# Building Generation Visualization

Interactive 3D visualization of buildings generated by `generate_buildings.py`.

Buildings are placed to avoid flyable corridors and each other.

In [1]:
import json
import subprocess
import tempfile
import os
import plotly.graph_objects as go
from typing import List, Dict


def generate_corridors(grid_size=400, num_ew=2, num_ns=2, width="20",
                       spacing="100", altitude_min=0, altitude_max=400, seed=None):
    """Run generate_corridors.py and return parsed corridors."""
    cmd = [
        "python3", "generate_corridors.py",
        "--grid-size", str(grid_size),
        "--num-ew", str(num_ew),
        "--num-ns", str(num_ns),
        "--width", str(width),
        "--spacing", str(spacing),
        "--altitude-min", str(altitude_min),
        "--altitude-max", str(altitude_max),
    ]
    if seed is not None:
        cmd.extend(["--seed", str(seed)])

    result = subprocess.run(cmd, capture_output=True, text=True)
    corridors = [json.loads(line) for line in result.stdout.strip().split('\n') if line]
    return corridors


def generate_buildings(corridors: List[Dict], grid_size=400, num_buildings=20,
                       width_x="30-50", width_y="30-50", height="50-150", seed=None):
    """Run generate_buildings.py and return parsed buildings."""
    # Write corridors to temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.ndjson', delete=False) as f:
        for c in corridors:
            json.dump(c, f)
            f.write('\n')
        corridors_file = f.name

    try:
        cmd = [
            "python3", "generate_buildings.py",
            "-c", corridors_file,
            "-n", str(num_buildings),
            "--grid-size", str(grid_size),
            "--width-x", str(width_x),
            "--width-y", str(width_y),
            "--height", str(height),
        ]
        if seed is not None:
            cmd.extend(["--seed", str(seed)])

        result = subprocess.run(cmd, capture_output=True, text=True)
        buildings = [json.loads(line) for line in result.stdout.strip().split('\n') if line]
        return buildings
    finally:
        os.unlink(corridors_file)


def create_corridor_mesh(corridor: Dict, grid_size: float, color: str, opacity: float = 0.2):
    """Create a 3D mesh for a corridor."""
    c = corridor["center"]
    w = corridor["width"] / 2
    z_min = corridor["altitude_min"]
    z_max = corridor["altitude_max"]

    if corridor["direction"] == "EW":
        x = [0, grid_size, grid_size, 0, 0, grid_size, grid_size, 0]
        y = [c-w, c-w, c+w, c+w, c-w, c-w, c+w, c+w]
    else:
        x = [c-w, c+w, c+w, c-w, c-w, c+w, c+w, c-w]
        y = [0, 0, grid_size, grid_size, 0, 0, grid_size, grid_size]

    z = [z_min, z_min, z_min, z_min, z_max, z_max, z_max, z_max]
    i = [0, 0, 4, 4, 0, 1, 0, 3, 1, 2, 4, 5]
    j = [1, 2, 5, 6, 1, 5, 3, 7, 2, 6, 5, 6]
    k = [2, 3, 6, 7, 4, 4, 4, 4, 5, 5, 1, 2]

    return go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        name=f"{corridor['direction']} corridor {corridor['id']}",
        hovertemplate=(
            f"Corridor {corridor['id']}<br>"
            f"Direction: {corridor['direction']}<br>"
            f"Center: {corridor['center']:.1f}m<br>"
            f"Width: {corridor['width']:.1f}m"
            "<extra></extra>"
        )
    )


def create_building_mesh(building: Dict, color: str = "gray", opacity: float = 0.8):
    """Create a 3D mesh for a building."""
    cx, cy = building["x"], building["y"]
    wx, wy = building["width_x"] / 2, building["width_y"] / 2
    h = building["height"]

    # 8 vertices of cuboid
    x = [cx-wx, cx+wx, cx+wx, cx-wx, cx-wx, cx+wx, cx+wx, cx-wx]
    y = [cy-wy, cy-wy, cy+wy, cy+wy, cy-wy, cy-wy, cy+wy, cy+wy]
    z = [0, 0, 0, 0, h, h, h, h]

    i = [0, 0, 4, 4, 0, 1, 0, 3, 1, 2, 4, 5]
    j = [1, 2, 5, 6, 1, 5, 3, 7, 2, 6, 5, 6]
    k = [2, 3, 6, 7, 4, 4, 4, 4, 5, 5, 1, 2]

    return go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        color=color,
        opacity=opacity,
        name=f"Building {building['id']}",
        hovertemplate=(
            f"Building {building['id']}<br>"
            f"Position: ({building['x']:.1f}, {building['y']:.1f})<br>"
            f"Size: {building['width_x']:.1f} x {building['width_y']:.1f}m<br>"
            f"Height: {building['height']:.1f}m"
            "<extra></extra>"
        )
    )


def visualize_urban_environment(corridors: List[Dict], buildings: List[Dict],
                                 grid_size: float = 400, title: str = "Urban Environment",
                                 show_corridors: bool = True):
    """Create interactive 3D visualization of corridors and buildings."""
    fig = go.Figure()

    # Add ground plane
    fig.add_trace(go.Mesh3d(
        x=[0, grid_size, grid_size, 0],
        y=[0, 0, grid_size, grid_size],
        z=[0, 0, 0, 0],
        i=[0, 0], j=[1, 2], k=[2, 3],
        color="lightgreen",
        opacity=0.5,
        name="Ground",
        hoverinfo="skip"
    ))

    # Add buildings
    for building in buildings:
        # Vary gray shade slightly
        gray = 100 + (building['id'] % 5) * 15
        color = f"rgb({gray}, {gray}, {gray})"
        fig.add_trace(create_building_mesh(building, color=color))

    # Add corridors (semi-transparent)
    if show_corridors:
        for corridor in corridors:
            color = "rgba(0, 100, 255, 0.15)" if corridor["direction"] == "EW" else "rgba(255, 50, 50, 0.15)"
            fig.add_trace(create_corridor_mesh(corridor, grid_size, color, opacity=0.15))

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title="X (East) [m]",
            yaxis_title="Y (North) [m]",
            zaxis_title="Altitude [m]",
            aspectmode="data",
            camera=dict(eye=dict(x=1.5, y=1.5, z=0.8))
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        height=700
    )

    return fig

## Example 1: Basic Urban Environment

2x2 corridor grid with 15 buildings.

In [2]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=42)
buildings = generate_buildings(corridors, num_buildings=15, seed=42)

print(f"Corridors: {len(corridors)}")
print(f"Buildings: {len(buildings)}")

fig = visualize_urban_environment(corridors, buildings, title="Basic Urban Environment")
fig.show()

Corridors: 4
Buildings: 15


## Example 2: Dense City Center

More corridors, more buildings, smaller building footprints.

In [3]:
corridors = generate_corridors(num_ew=3, num_ns=3, width="15", spacing="100", seed=1)
buildings = generate_buildings(
    corridors,
    num_buildings=30,
    width_x="20-35",
    width_y="20-35",
    height="60-180",
    seed=1
)

print(f"Corridors: {len(corridors)}")
print(f"Buildings: {len(buildings)}")

fig = visualize_urban_environment(corridors, buildings, title="Dense City Center")
fig.show()

Corridors: 6
Buildings: 30


## Example 3: Low-Rise Suburban

Wider corridors, shorter buildings, more spread out.

In [4]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="30", spacing="150", seed=5)
buildings = generate_buildings(
    corridors,
    num_buildings=12,
    width_x="40-60",
    width_y="40-60",
    height="20-50",
    seed=5
)

print(f"Corridors: {len(corridors)}")
print(f"Buildings: {len(buildings)}")

fig = visualize_urban_environment(corridors, buildings, title="Low-Rise Suburban")
fig.show()

Corridors: 4
Buildings: 12


## Example 4: Skyscraper District

Tall, narrow buildings with tight corridors.

In [5]:
corridors = generate_corridors(num_ew=3, num_ns=3, width="20", spacing="90", seed=10)
buildings = generate_buildings(
    corridors,
    num_buildings=25,
    width_x="25-40",
    width_y="25-40",
    height="100-250",
    seed=10
)

print(f"Corridors: {len(corridors)}")
print(f"Buildings: {len(buildings)}")

fig = visualize_urban_environment(corridors, buildings, title="Skyscraper District")
fig.show()

Corridors: 6
Buildings: 25


## Example 5: Randomized Corridors

Variable corridor widths and spacing.

In [6]:
corridors = generate_corridors(num_ew=3, num_ns=2, width="15-25", spacing="80-130", seed=99)
buildings = generate_buildings(
    corridors,
    num_buildings=20,
    width_x="30-50",
    width_y="30-50",
    height="50-150",
    seed=99
)

print(f"Corridors: {len(corridors)}")
for c in corridors:
    print(f"  {c['direction']} @ {c['center']:.1f}m, width={c['width']:.1f}m")

fig = visualize_urban_environment(corridors, buildings, title="Randomized Corridors")
fig.show()

Corridors: 5
  EW @ 100.2m, width=17.0m
  EW @ 189.1m, width=17.5m
  EW @ 307.1m, width=17.5m
  NS @ 99.2m, width=21.8m
  NS @ 206.1m, width=24.4m


## Example 6: Same Corridors, Different Building Seeds

Shows how the same corridor layout can have different building arrangements.

In [7]:
# Fixed corridors
corridors = generate_corridors(num_ew=2, num_ns=2, width="20", spacing="120", seed=42)

for building_seed in [1, 42, 99]:
    buildings = generate_buildings(corridors, num_buildings=15, seed=building_seed)
    fig = visualize_urban_environment(
        corridors, buildings,
        title=f"Same Corridors, Building Seed={building_seed}"
    )
    fig.show()

## Example 7: Buildings Only (No Corridor Overlay)

Cleaner view with corridors hidden.

In [8]:
corridors = generate_corridors(num_ew=2, num_ns=2, width="25", spacing="130", seed=7)
buildings = generate_buildings(corridors, num_buildings=18, height="40-120", seed=7)

fig = visualize_urban_environment(
    corridors, buildings,
    title="Urban Environment (Corridors Hidden)",
    show_corridors=False
)
fig.show()

## Example 8: Maximum Density Test

Try to pack many buildings - some may fail to place.

In [9]:
corridors = generate_corridors(num_ew=4, num_ns=4, width="15", spacing="80", seed=50)
buildings = generate_buildings(
    corridors,
    num_buildings=50,  # Request many
    width_x="20-30",
    width_y="20-30",
    height="50-100",
    seed=50
)

print(f"Requested 50 buildings, placed {len(buildings)}")

fig = visualize_urban_environment(corridors, buildings, title="Maximum Density Test")
fig.show()

Requested 50 buildings, placed 50
